In [ ]:
import os
import pandas as pd
#import psycopg2 as pg2
import warnings
import pyodbc

warnings.filterwarnings("ignore")

: 

In [6]:
import json

with open('config.json') as f:
    config = json.load(f)

server = config['server']
database = config['database']

conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')

In [3]:
cursor = conn.cursor()

query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'"
cursor.execute(query)

tables = cursor.fetchall()
for table in tables:
    print(table)

('olympic_data',)
('athletes_roles',)
('host_cities',)
('countries_noc',)
('temp_table',)


In [4]:
host_cities_df = pd.read_csv("data/host_cities.csv")
host_cities_df

,Year,Season,Game,Host_City
0,1896,Summer,1896 Summer Olympics,Athina
1,1900,Summer,1900 Summer Olympics,Paris
2,1904,Summer,1904 Summer Olympics,St. Louis
3,1908,Summer,1908 Summer Olympics,London
4,1912,Summer,1912 Summer Olympics,Stockholm
...,...,...,...,...
57,2010,Winter,2010 Winter Olympics,Vancouver
58,2014,Winter,2014 Winter Olympics,Sochi
59,2018,Winter,2018 Winter Olympics,PyeongChang
60,2022,Winter,2022 Winter Olympics,Beijing


In [6]:
columns = ', '.join(host_cities_df.columns.tolist())
placeholders = ', '.join(['?'] * len(host_cities_df.columns))
sql = f"INSERT INTO {'olympics.host_cities'} ({columns}) VALUES ({placeholders})"

data_list = host_cities_df.to_records(index=False).tolist()

try:
    cursor = conn.cursor()
    cursor.executemany(sql, data_list)
    conn.commit()
    print(f"Successfully inserted {len(data_list)} rows!")
except pyodbc.Error as e:
    print(f"Error during bulk insertion: {e}")


Successfully inserted 62 rows!


In [7]:
countries_noc_df = pd.read_csv("data/countries_noc.csv")
countries_noc_df

,noc,country
0,AFG,Afghanistan
1,ALB,Albania
2,ALG,Algeria
3,ASA,American Samoa
4,AND,Andorra
...,...,...
229,WIF,West Indies Federation
230,YEM,Yemen
231,YUG,Yugoslavia
232,ZAM,Zambia


In [9]:
columns = ', '.join(countries_noc_df.columns.tolist())
placeholders = ', '.join(['?'] * len(countries_noc_df.columns))
sql = f"INSERT INTO {'olympics.countries_noc'} ({columns}) VALUES ({placeholders})"

data_list = countries_noc_df.to_records(index=False).tolist()

try:
    cursor = conn.cursor()
    cursor.executemany(sql, data_list)
    conn.commit()
    print(f"Successfully inserted {len(data_list)} rows!")
except pyodbc.Error as e:
    print(f"Error during bulk insertion: {e}")


Successfully inserted 234 rows!


In [10]:
athletes_roles_df = pd.read_csv("data/athletes_roles.csv")
athletes_roles_df

,id,name,roles
0,111427,Cui Peng,Competed in Olympic Games
1,97418,Muzaffer Selçuk,Competed in Olympic Games
2,18579,Georges Breuille,Competed in Olympic Games
3,55113,Ángel Sposato,Competed in Olympic Games
4,143768,Udayan Mane,Competed in Olympic Games
...,...,...,...
155932,37771,Ralf Brudel,Competed in Olympic Games
155933,145001,Lee Hye-In,Competed in Olympic Games
155934,67556,Lauri Eskola,Competed in Olympic Games
155935,902192,Liu Chiu-Lung,Competed in Olympic Games (non-medal events)


In [11]:
columns = ', '.join(athletes_roles_df.columns.tolist())
placeholders = ', '.join(['?'] * len(athletes_roles_df.columns))
sql = f"INSERT INTO {'olympics.athletes_roles'} ({columns}) VALUES ({placeholders})"

data_list = athletes_roles_df.to_records(index=False).tolist()

try:
    cursor = conn.cursor()
    cursor.executemany(sql, data_list)
    conn.commit()
    print(f"Successfully inserted {len(data_list)} rows!")
except pyodbc.Error as e:
    print(f"Error during bulk insertion: {e}")


Successfully inserted 155937 rows!


In [3]:
athletes_data_df = pd.read_csv("data/athletes_data.csv")
athletes_data_df

,id,name,gender,born,died,height,weight,team,game,noc,sport,event,position,medal
0,111427,Cui Peng,Male,31 May 1987,NaN,179 cm,72 kg,People's Republic of China,2008 Summer Olympics,CHN,Football (Football),"('Football (Football), Football, Men(Olympic)',)","('13',)",NaN
1,97418,Muzaffer Selçuk,Male,12 August 1961,NaN,180 cm,74 kg,Türkiye,1984 Winter Olympics,TUR,Cross Country Skiing (Skiing),"('Cross Country Skiing (Skiing), 15 kilometres...","('DNF',)",NaN
2,18579,Georges Breuille,Male,NaN,NaN,NaN,NaN,France,1920 Summer Olympics,FRA,Hockey,"('Hockey, Hockey, Men(Olympic)',)","('4',)",NaN
3,55113,Ángel Sposato,Male,17 May 1922,NaN,NaN,74 kg,Argentina,1952 Summer Olympics,ARG,Weightlifting,"('Weightlifting, Middleweight, Men(Olympic)',)","('7',)",NaN
4,143768,Udayan Mane,Male,24 February 1991,NaN,192 cm,NaN,India,2020 Summer Olympics,IND,Golf,"('Golf, Individual, Men(Olympic)',)","('56',)",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321423,145001,Lee Hye-In,Female,16 January 1995,NaN,NaN,NaN,Republic of Korea,2020 Summer Olympics,KOR,Fencing,"('Fencing, Épée, Team, Women(Olympic)',)","('2',)",Silver
321424,67556,Lauri Eskola,Male,16 November 1886,7 January 1958,NaN,NaN,Finland,1912 Summer Olympics,FIN,Athletics,"('Athletics, Cross-Country, Individual, Men(Ol...","('4',)",NaN
321425,67556,Lauri Eskola,Male,16 November 1886,7 January 1958,NaN,NaN,Finland,1912 Summer Olympics,FIN,Athletics,"('Athletics, Cross-Country, Team, Men(Olympic)',)","('2',)",Silver
321426,902192,Liu Chiu-Lung,Male,18 September 1956,NaN,NaN,NaN,Chinese Taipei,1984 Summer Olympics,TPE,Baseball (Baseball/Softball),"('Baseball (Baseball/Softball), Baseball, Men(...","('3',)",NaN


In [4]:
athletes_data_df = athletes_data_df.replace(pd.NA, 'NULL')
athletes_data_df

,id,name,gender,born,died,height,weight,team,game,noc,sport,event,position,medal
0,111427,Cui Peng,Male,31 May 1987,NULL,179 cm,72 kg,People's Republic of China,2008 Summer Olympics,CHN,Football (Football),"('Football (Football), Football, Men(Olympic)',)","('13',)",NULL
1,97418,Muzaffer Selçuk,Male,12 August 1961,NULL,180 cm,74 kg,Türkiye,1984 Winter Olympics,TUR,Cross Country Skiing (Skiing),"('Cross Country Skiing (Skiing), 15 kilometres...","('DNF',)",NULL
2,18579,Georges Breuille,Male,NULL,NULL,NULL,NULL,France,1920 Summer Olympics,FRA,Hockey,"('Hockey, Hockey, Men(Olympic)',)","('4',)",NULL
3,55113,Ángel Sposato,Male,17 May 1922,NULL,NULL,74 kg,Argentina,1952 Summer Olympics,ARG,Weightlifting,"('Weightlifting, Middleweight, Men(Olympic)',)","('7',)",NULL
4,143768,Udayan Mane,Male,24 February 1991,NULL,192 cm,NULL,India,2020 Summer Olympics,IND,Golf,"('Golf, Individual, Men(Olympic)',)","('56',)",NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321423,145001,Lee Hye-In,Female,16 January 1995,NULL,NULL,NULL,Republic of Korea,2020 Summer Olympics,KOR,Fencing,"('Fencing, Épée, Team, Women(Olympic)',)","('2',)",Silver
321424,67556,Lauri Eskola,Male,16 November 1886,7 January 1958,NULL,NULL,Finland,1912 Summer Olympics,FIN,Athletics,"('Athletics, Cross-Country, Individual, Men(Ol...","('4',)",NULL
321425,67556,Lauri Eskola,Male,16 November 1886,7 January 1958,NULL,NULL,Finland,1912 Summer Olympics,FIN,Athletics,"('Athletics, Cross-Country, Team, Men(Olympic)',)","('2',)",Silver
321426,902192,Liu Chiu-Lung,Male,18 September 1956,NULL,NULL,NULL,Chinese Taipei,1984 Summer Olympics,TPE,Baseball (Baseball/Softball),"('Baseball (Baseball/Softball), Baseball, Men(...","('3',)",NULL


In [7]:
columns = ', '.join(athletes_data_df.columns.tolist())
placeholders = ', '.join(['?'] * len(athletes_data_df.columns))
sql = f"INSERT INTO {'olympics.athlete_data'} ({columns}) VALUES ({placeholders})"

data_list = athletes_data_df.to_records(index=False).tolist()

try:
    cursor = conn.cursor()
    cursor.executemany(sql, data_list)
    conn.commit()
    print(f"Successfully inserted {len(data_list)} rows!")
except pyodbc.Error as e:
    print(f"Error during bulk insertion: {e}")


Successfully inserted 321428 rows!


In [7]:
pd.read_sql("""
SELECT DISTINCT born
FROM olympics.athlete_data
WHERE born NOT LIKE '[0-9]%' AND born != 'NULL'
""", conn)

,born
0,(1916 or 1917)
1,(1926 or 1927)
2,(1945 or 1946)
3,(1946 or 1947)
4,(1949 or 1950)
...,...
143,July 1882
144,June 1914
145,March 1868
146,May 1863


In [8]:
cursor.execute("""
    UPDATE olympics.athlete_data
SET
    height = CASE WHEN height = 'NULL' THEN NULL ELSE CAST(LEFT(height, 2) AS INT) END,
    weight = CASE WHEN weight = 'NULL' THEN NULL ELSE CAST(LEFT(weight, 2) AS INT) END;
               """)

In [9]:
conn.commit()

In [10]:
cursor.execute("""
    ALTER TABLE olympics.athlete_data
    ADD birth_year INT NULL, death_year INT NULL; 
               """)

In [11]:
conn.commit()

In [12]:
cursor.execute("""
    ALTER TABLE olympics.athlete_data
    ALTER COLUMN birth_year NVARCHAR(MAX) NULL;
    """)

In [13]:
cursor.execute("""
    ALTER TABLE olympics.athlete_data
    ALTER COLUMN death_year NVARCHAR(MAX) NULL;
    """)

In [14]:
cursor.execute("""
    UPDATE olympics.athlete_data
SET
    birth_year = CASE WHEN born = 'NULL' THEN NULL ELSE CAST(SUBSTRING(born, PATINDEX('%[0-9][0-9][0-9][0-9]%', born), 4) AS INT) END,
    death_year = CASE WHEN died = 'NULL' THEN NULL 
               ELSE SUBSTRING(died, PATINDEX('%[0-9][0-9][0-9][0-9]%', died), 4) END;
               """)

In [15]:
pd.read_sql("""
SELECT *
FROM olympics.athlete_data
where death_year LIKE '(%'
""", conn)

,id,name,gender,born,died,height,weight,team,game,noc,sport,event,position,medal,birth_year,death_year
0,2304424,Desiré Wuyts,Male,10 March 1901,"(Deceased, details unknown)",None,None,Belgium,1920 Summer Olympics,BEL,Tug-Of-War,"('Tug-Of-War, Tug-Of-War, Men(Olympic)',)","('3',)",Bronze,1901,(De
1,15780,Rusi Mulla Feroze,Male,1922,"(Deceased, details unknown)",None,None,India,1948 Summer Olympics,IND,Cycling Track (Cycling),"('Cycling Track (Cycling), Sprint, Men(Olympic...","('2 h2 r2/6',)",NULL,1922,(De
2,43674,Bernardo San Juan,Male,12 March 1924,"(Deceased, details unknown)",16,62,Philippines,1960 Summer Olympics,PHI,Shooting,"('Shooting, Small-Bore Rifle, Three Positions,...","('51',)",NULL,1924,(De
3,43674,Bernardo San Juan,Male,12 March 1924,"(Deceased, details unknown)",16,62,Philippines,1964 Summer Olympics,PHI,Shooting,"('Shooting, Free Rifle, Three Positions, 300 m...","('26',)",NULL,1924,(De
4,43674,Bernardo San Juan,Male,12 March 1924,"(Deceased, details unknown)",16,62,Philippines,1968 Summer Olympics,PHI,Shooting,"('Shooting, Free Rifle, Three Positions, 300 m...","('28',)",NULL,1924,(De
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,29314,Marica Ivandekić,Female,25 August 1925,"(Deceased, details unknown)",None,None,Yugoslavia,1952 Summer Olympics,YUG,Artistic Gymnastics (Gymnastics),"('Artistic Gymnastics (Gymnastics), Uneven Bar...","('=80',)",NULL,1925,(De
98,29314,Marica Ivandekić,Female,25 August 1925,"(Deceased, details unknown)",None,None,Yugoslavia,1952 Summer Olympics,YUG,Artistic Gymnastics (Gymnastics),"('Artistic Gymnastics (Gymnastics), Balance Be...","('72',)",NULL,1925,(De
99,8280,Jozef Töre,Male,18 June 1933,"(Deceased, details unknown)",16,60,Czechoslovakia,1960 Summer Olympics,TCH,Boxing,"('Boxing, Lightweight, Men(Olympic)',)","('=17',)",NULL,1933,(De
100,15641,Mesfen Tesfaye,Male,13 January 1935,"(Deceased, details unknown)",None,None,Ethiopia,1956 Summer Olympics,ETH,Cycling Road (Cycling),"('Cycling Road (Cycling), Road Race, Individua...","('36',)",NULL,1935,(De


In [16]:
cursor.execute("""
    UPDATE olympics.athlete_data
SET
    birth_year = NULL, 
    death_year = NULL
               """)

In [17]:
cursor.execute("""
    ALTER TABLE olympics.athlete_data
    ALTER COLUMN birth_year INT NULL;
    """)

In [18]:
cursor.execute("""
    ALTER TABLE olympics.athlete_data
    ALTER COLUMN death_year INT NULL;
    """)

In [19]:
cursor.execute("""
    UPDATE olympics.athlete_data
SET
    birth_year = CASE WHEN born = 'NULL' THEN NULL ELSE CAST(SUBSTRING(born, PATINDEX('%[0-9][0-9][0-9][0-9]%', born), 4) AS INT) END,
    death_year = CASE WHEN died = 'NULL' THEN NULL 
               WHEN died = '(Deceased, details unknown)' THEN NULL
               WHEN died = '(deceased, DPOD unknown)' THEN NULL
               WHEN died = '(deceased)' THEN NULL
               WHEN died = '(' THEN NULL
               WHEN died = '(Died)' THEN NULL
               WHEN died = '(Died' THEN NULL
               WHEN died = '(Died' THEN NULL
               WHEN died = '(At the age of 62)' THEN NULL
               ELSE CAST(SUBSTRING(died, PATINDEX('%[0-9][0-9][0-9][0-9]%', died), 4) AS INT) END;
               """)

In [20]:
pd.read_sql("""
SELECT *
FROM olympics.athlete_data
""", conn)

,id,name,gender,born,died,height,weight,team,game,noc,sport,event,position,medal,birth_year,death_year
0,110823,Alidad Saveh Shemshaki,Male,23 July 1972,NULL,17,None,Islamic Republic of Iran,2006 Winter Olympics,IRI,Alpine Skiing (Skiing),"('Alpine Skiing (Skiing), Giant Slalom, Men(Ol...","('36',)",NULL,1972.0,NaN
1,110823,Alidad Saveh Shemshaki,Male,23 July 1972,NULL,17,None,Islamic Republic of Iran,2006 Winter Olympics,IRI,Alpine Skiing (Skiing),"('Alpine Skiing (Skiing), Slalom, Men(Olympic)',)","('41',)",NULL,1972.0,NaN
2,60246,Josef Zeman,Male,24 December 1925,2003,None,None,Czechoslovakia,1952 Summer Olympics,TCH,Wrestling,"('Wrestling, Flyweight, Greco-Roman, Men(Olymp...","('AC',)",NULL,1925.0,2003.0
3,2503190,Ákos Kalmár,Male,11 January 2000,NULL,18,84,Hungary,2018 Summer Youth Olympics,HUN,Swimming (Aquatics),"('Swimming (Aquatics), 200 metres Freestyle, B...","('22',)",NULL,2000.0,NaN
4,2503190,Ákos Kalmár,Male,11 January 2000,NULL,18,84,Hungary,2018 Summer Youth Olympics,HUN,Swimming (Aquatics),"('Swimming (Aquatics), 400 metres Freestyle, B...","('5',)",NULL,2000.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321423,65176,Jacques Delelienne,Male,25 November 1928,3 February 2020,None,None,Belgium,1952 Summer Olympics,BEL,Athletics,"('Athletics, High Jump, Men(Olympic)',)","('=9',)",NULL,1928.0,2020.0
321424,111283,Yuto Nagatomo,Male,12 September 1986,NULL,16,65,Japan,2008 Summer Olympics,JPN,Football (Football),"('Football (Football), Football, Men(Olympic)',)","('15',)",NULL,1986.0,NaN
321425,35389,Ben Dodwell,Male,17 April 1972,NULL,20,98,Australia,1992 Summer Olympics,AUS,Rowing,"('Rowing, Eights, Men(Olympic)',)","('5',)",NULL,1972.0,NaN
321426,35389,Ben Dodwell,Male,17 April 1972,NULL,20,98,Australia,1996 Summer Olympics,AUS,Rowing,"('Rowing, Eights, Men(Olympic)',)","('6',)",NULL,1972.0,NaN


In [21]:
conn.commit()

In [22]:
pd.read_sql("""
               SELECT COUNT(*)
FROM olympics.athlete_data
WHERE birth_year > death_year;
               """, conn)

,
0,0


In [23]:
conn.commit()

In [24]:
pd.read_sql(""" 
select distinct * from olympics.athlete_data           
""", conn)

,id,name,gender,born,died,height,weight,team,game,noc,sport,event,position,medal,birth_year,death_year
0,4,Jacques Brugnon,Male,11 May 1895,20 March 1978,16,64,France,1920 Summer Olympics,FRA,Tennis,"('Tennis, Doubles, Men(Olympic)',)","('4',)",NULL,1895.0,1978.0
1,1,Jean-François Blanchy,Male,12 December 1886,2 October 1960,None,None,France,1912 Summer Olympics,FRA,Tennis,"('Tennis, Doubles, Men(Olympic)',)","('DNS',)",NULL,1886.0,1960.0
2,1,Jean-François Blanchy,Male,12 December 1886,2 October 1960,None,None,France,1912 Summer Olympics,FRA,Tennis,"('Tennis, Singles, Men(Olympic)',)","('=17',)",NULL,1886.0,1960.0
3,3,Jean Borotra,Male,13 August 1898,17 July 1994,18,76,France,1924 Summer Olympics,FRA,Tennis,"('Tennis, Doubles, Mixed(Olympic)',)","('=15',)",NULL,1898.0,1994.0
4,4,Jacques Brugnon,Male,11 May 1895,20 March 1978,16,64,France,1920 Summer Olympics,FRA,Tennis,"('Tennis, Singles, Men(Olympic)',)","('=9',)",NULL,1895.0,1978.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320033,2700060,Lombière,Male,NULL,NULL,None,None,France,1900 Summer Olympics,FRA,Athletics,"('Athletics, 100 metres, Professionals, Handic...","('2 h5 r1/2',)",NULL,NaN,NaN
320034,2700069,G. L. Gerarden,Male,NULL,NULL,None,None,United States,1900 Summer Olympics,USA,Athletics,"('Athletics, 400 metres, Professionals, Men(Ol...","('AC',)",NULL,NaN,NaN
320035,2800008,Richard Kimball,Male,29 January 1867,6 November 1915,None,None,United States,1904 Summer Olympics,USA,Golf,"('Golf, Individual, Men(Olympic)',)","('DNS',)",NULL,1867.0,1915.0
320036,3100000,Francis Conry,Male,30 November 1874,30 September 1938,None,None,Great Britain,1900 Summer Olympics,GBR,Cricket,"('Cricket, Cricket, Men(Olympic)',)","('DNS',)",NULL,1874.0,1938.0


In [25]:
cursor.execute("""
    SELECT DISTINCT *
INTO temp_table
FROM olympics.athlete_data;
"""
)


In [26]:
pd.read_sql(""" select * from temp_table """, conn)

,id,name,gender,born,died,height,weight,team,game,noc,sport,event,position,medal,birth_year,death_year
0,1,Jean-François Blanchy,Male,12 December 1886,2 October 1960,None,None,France,1920 Summer Olympics,FRA,Tennis,"('Tennis, Doubles, Men(Olympic)',)","('4',)",NULL,1886.0,1960.0
1,1,Jean-François Blanchy,Male,12 December 1886,2 October 1960,None,None,France,1920 Summer Olympics,FRA,Tennis,"('Tennis, Singles, Men(Olympic)',)","('=32',)",NULL,1886.0,1960.0
2,4,Jacques Brugnon,Male,11 May 1895,20 March 1978,16,64,France,1924 Summer Olympics,FRA,Tennis,"('Tennis, Doubles, Men(Olympic)',)","('2',)",Silver,1895.0,1978.0
3,5,Albert Canet,Male,17 April 1878,25 July 1930,None,None,France,1912 Summer Olympics,FRA,Tennis,"('Tennis, Doubles, Men(Olympic)',)","('3',)",Bronze,1878.0,1930.0
4,5,Albert Canet,Male,17 April 1878,25 July 1930,None,None,France,1912 Summer Olympics,FRA,Tennis,"('Tennis, Doubles, Mixed(Olympic)',)","('3',)",Bronze,1878.0,1930.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320033,3100004,Lucius Vicary,Male,2 December 1881,29 March 1953,None,None,Great Britain,1900 Summer Olympics,GBR,Cricket,"('Cricket, Cricket, Men(Olympic)',)","('DNS',)",NULL,1881.0,1953.0
320034,3100005,Robert Roles,Male,6 May 1868,20 May 1928,None,None,Great Britain,1900 Summer Olympics,GBR,Cricket,"('Cricket, Cricket, Men(Olympic)',)","('DNS',)",NULL,1868.0,1928.0
320035,3100006,Andrew Mackie,Male,31 May 1882,18 October 1945,None,None,Great Britain,1900 Summer Olympics,GBR,Cricket,"('Cricket, Cricket, Men(Olympic)',)","('DNS',)",NULL,1882.0,1945.0
320036,3100008,James Wilcocks,Male,22 January 1872,28 September 1943,None,None,Great Britain,1900 Summer Olympics,GBR,Cricket,"('Cricket, Cricket, Men(Olympic)',)","('DNS',)",NULL,1872.0,1943.0


In [27]:
cursor.execute("""
    TRUNCATE TABLE olympics.athlete_data;
"""
)


In [28]:
cursor.execute("""
    INSERT INTO olympics.athlete_data
SELECT *
FROM temp_table;
"""
)


In [29]:
pd.read_sql(""" select * from  olympics.athlete_data """, conn)

,id,name,gender,born,died,height,weight,team,game,noc,sport,event,position,medal,birth_year,death_year
0,98497,André Usborne,Male,16 October 1952,NULL,16,66,Great Britain,1984 Winter Olympics,GBR,Luge,"('Luge, Singles, Men(Olympic)',)","('27',)",NULL,1952.0,NaN
1,98498,Ruedi Uster,Male,4 September 1941,NULL,17,75,Switzerland,1968 Winter Olympics,SUI,Speed Skating (Skating),"('Speed Skating (Skating), 500 metres, Men(Oly...","('43',)",NULL,1941.0,NaN
2,98500,Taner Üstündağ,Male,28 October 1971,NULL,None,None,Türkiye,1992 Winter Olympics,TUR,Alpine Skiing (Skiing),"('Alpine Skiing (Skiing), Giant Slalom, Men(Ol...","('60',)",NULL,1971.0,NaN
3,98501,Stanisław Ustupski,Male,15 November 1966,NULL,17,63,Poland,1992 Winter Olympics,POL,Nordic Combined (Skiing),"('Nordic Combined (Skiing), Individual, Men(Ol...","('8',)",NULL,1966.0,NaN
4,98504,Peter Utzschneider,Male,6 March 1946,NULL,18,87,West Germany,1968 Winter Olympics,FRG,Bobsleigh (Bobsleigh),"('Bobsleigh (Bobsleigh), Four, Men(Olympic)',)","('9',)",NULL,1946.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320033,98488,Gundars Upenieks,Male,31 March 1971,NULL,18,80,Latvia,1998 Winter Olympics,LAT,Biathlon,"('Biathlon, 4 × 7.5 kilometres Relay, Men(Olym...","('6',)",NULL,1971.0,NaN
320034,98492,Petr Urban,Male,8 August 1960,NULL,None,None,Czechoslovakia,1992 Winter Olympics,TCH,Luge,"('Luge, Doubles, Open(Olympic)',)","('15',)",NULL,1960.0,NaN
320035,98492,Petr Urban,Male,8 August 1960,NULL,None,None,Czechoslovakia,1992 Winter Olympics,TCH,Luge,"('Luge, Singles, Men(Olympic)',)","('19',)",NULL,1960.0,NaN
320036,98493,Roland Urban,Male,29 March 1940,NULL,17,78,Czechoslovakia,1968 Winter Olympics,TCH,Luge,"('Luge, Doubles, Open(Olympic)',)","('12',)",NULL,1940.0,NaN


In [30]:
conn.commit()

In [31]:
cursor.execute("""
    UPDATE olympics.athlete_data
SET position = REPLACE(REPLACE(REPLACE(REPLACE(REPLACE(position, '(', ''), ')', ''), '''', ''), ',', ''), '=', '');
               """)

In [32]:
pd.read_sql(""" select * from  olympics.athlete_data """, conn)

,id,name,gender,born,died,height,weight,team,game,noc,sport,event,position,medal,birth_year,death_year
0,98497,André Usborne,Male,16 October 1952,NULL,16,66,Great Britain,1984 Winter Olympics,GBR,Luge,"('Luge, Singles, Men(Olympic)',)",27,NULL,1952.0,NaN
1,98498,Ruedi Uster,Male,4 September 1941,NULL,17,75,Switzerland,1968 Winter Olympics,SUI,Speed Skating (Skating),"('Speed Skating (Skating), 500 metres, Men(Oly...",43,NULL,1941.0,NaN
2,98500,Taner Üstündağ,Male,28 October 1971,NULL,None,None,Türkiye,1992 Winter Olympics,TUR,Alpine Skiing (Skiing),"('Alpine Skiing (Skiing), Giant Slalom, Men(Ol...",60,NULL,1971.0,NaN
3,98501,Stanisław Ustupski,Male,15 November 1966,NULL,17,63,Poland,1992 Winter Olympics,POL,Nordic Combined (Skiing),"('Nordic Combined (Skiing), Individual, Men(Ol...",8,NULL,1966.0,NaN
4,98504,Peter Utzschneider,Male,6 March 1946,NULL,18,87,West Germany,1968 Winter Olympics,FRG,Bobsleigh (Bobsleigh),"('Bobsleigh (Bobsleigh), Four, Men(Olympic)',)",9,NULL,1946.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320033,98488,Gundars Upenieks,Male,31 March 1971,NULL,18,80,Latvia,1998 Winter Olympics,LAT,Biathlon,"('Biathlon, 4 × 7.5 kilometres Relay, Men(Olym...",6,NULL,1971.0,NaN
320034,98492,Petr Urban,Male,8 August 1960,NULL,None,None,Czechoslovakia,1992 Winter Olympics,TCH,Luge,"('Luge, Doubles, Open(Olympic)',)",15,NULL,1960.0,NaN
320035,98492,Petr Urban,Male,8 August 1960,NULL,None,None,Czechoslovakia,1992 Winter Olympics,TCH,Luge,"('Luge, Singles, Men(Olympic)',)",19,NULL,1960.0,NaN
320036,98493,Roland Urban,Male,29 March 1940,NULL,17,78,Czechoslovakia,1968 Winter Olympics,TCH,Luge,"('Luge, Doubles, Open(Olympic)',)",12,NULL,1940.0,NaN


In [33]:
pd.read_sql("""
SELECT *
FROM olympics.athlete_data
WHERE position LIKE '%r%';
""", conn)

,id,name,gender,born,died,height,weight,team,game,noc,sport,event,position,medal,birth_year,death_year
0,98634,Gilberto Viadana,Male,9 February 1973,NULL,17,72,Italy,1992 Winter Olympics,ITA,Figure Skating (Skating),"('Figure Skating (Skating), Singles, Men(Olymp...",DNS r2/2,NULL,1973.0,NaN
1,99143,Osman Yüce,Male,21 April 1929,26 January 1965,None,None,Türkiye,1964 Winter Olympics,TUR,Alpine Skiing (Skiing),"('Alpine Skiing (Skiing), Slalom, Men(Olympic)',)",AC r1/2,NULL,1929.0,1965.0
2,100023,Luan Zhengrong,Female,7 April 1974,NULL,16,54,People's Republic of China,2002 Winter Olympics,CHN,Cross Country Skiing (Skiing),"('Cross Country Skiing (Skiing), 5/5 kilometre...",65 r1/2,NULL,1974.0,NaN
3,100153,Lee Kyu-Hyun,Male,15 October 1980,NULL,18,70,Republic of Korea,2002 Winter Olympics,KOR,Figure Skating (Skating),"('Figure Skating (Skating), Singles, Men(Olymp...",28 r1/2,NULL,1980.0,NaN
4,100157,Patrick Schmit,Male,1 November 1974,NULL,16,57,Luxembourg,1998 Winter Olympics,LUX,Figure Skating (Skating),"('Figure Skating (Skating), Singles, Men(Olymp...",29 r1/2,NULL,1974.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48953,97138,Luis Sánchez,Male,12 December 1940,NULL,16,70,Spain,1964 Winter Olympics,ESP,Alpine Skiing (Skiing),"('Alpine Skiing (Skiing), Slalom, Men(Olympic)',)",26 r1/2,NULL,1940.0,NaN
48954,97201,Ali Saveh,Male,1949,NULL,17,72,Islamic Republic of Iran,1968 Winter Olympics,IRI,Alpine Skiing (Skiing),"('Alpine Skiing (Skiing), Slalom, Men(Olympic)',)",2 h5 r2/3,NULL,1949.0,NaN
48955,97312,Karl Schranz,Male,18 November 1938,NULL,17,73,Austria,1968 Winter Olympics,AUT,Alpine Skiing (Skiing),"('Alpine Skiing (Skiing), Slalom, Men(Olympic)',)",AC r3/3,NULL,1938.0,NaN
48956,97540,Eduardo Silva,Male,7 October 1928,July 2017,None,None,Chile,1952 Winter Olympics,CHI,Alpine Skiing (Skiing),"('Alpine Skiing (Skiing), Slalom, Men(Olympic)',)",56 r1/2,NULL,1928.0,2017.0


In [34]:
conn.commit()